# Install SlayerPytorch on Colab
After the installations the runtime needs to be restarted. 
```
exit()
```
Will restart the runtime without deleting files. The runtime will automatically start. And if you press "run all" the run is not interrupted and works till the end.

In [ ]:
!git clone https://github.com/bamsumit/slayerPytorch
!pip install dv
!pip install ninja
exit()

In [ ]:
%cd slayerPytorch/
!python setup.py install
exit()

Test to verify if everything went well with the installation

In [ ]:
%cd slayerPytorch/test/
!python -m  unittest

# Get the dataset
Download the dataset from [here](https://ibm.ent.box.com/s/3hiq58ww1pbbjrinh367ykfdf60xsfm8).
Be careful to select the right path when you unzip the files.

In [ ]:
!unzip './DVSGesturedataset.zip' -d /content/slayerPytorch

# Pre-process the dataset
Every sample contains all the actions performed in sequence by a subject. This program is able to divide the samples into smaller ones, containing single actions.


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import slayerSNN as snn
from dv import LegacyAedatFile

path = '/content/slayerPytorch/DVSGesturedataset/DvsGesture/'

actionName = [
    'hand_clapping',
    'right_hand_wave',
    'left_hand_wave',
    'right_arm_clockwise',
    'right_arm_counter_clockwise',
    'left_arm_clockwise', 
    'left_arm_counter_clockwise',
    'arm_roll',
    'air_drums',
    'air_guitar',
    'other_gestures',
]

def readAedatEvent(filename):
    xEvent = []
    yEvent = []
    pEvent = []
    tEvent = []
    with LegacyAedatFile(filename) as f:
        for event in f:
            xEvent.append(event.x)
            yEvent.append(event.y)
            pEvent.append(event.polarity)
            tEvent.append(event.timestamp/1000)

    return xEvent, yEvent, pEvent, tEvent

    def splitData(filename, path):
    x,y,p,t = readAedatEvent(path+filename+'.aedat')
    labels = np.loadtxt(path + filename + '_labels.csv', delimiter=',', skiprows=1)
    labels[:,0]  -= 1
    labels[:,1:]
    if not os.path.isdir(path+ 'data/' + filename):
        os.makedirs(os.path.join(path,'data/' + filename))
    lastAction = 100
    for action, tst, ten in labels:
        if action == lastAction:    continue # This is to ignore second arm_roll samples
        print(actionName[int(action)])
        ind = (t >= tst/1000) & (t < ten/1000)
        ind_in = np.argmax(ind)
        ind_end = ind_in+np.argmin(ind[(ind.argmax()):-1])
        if ind_end == ind_in:
          ind_end = 0
        TD = snn.io.event(x[ind_in:ind_end-1], y[ind_in:ind_end-1], p[ind_in:ind_end-1], (t[ind_in:ind_end-1] - tst/1000))
        # snn.io.showTD(TD)
        lastAction = action

        snn.io.encodeNpSpikes(path+'data/'+ filename + '/{:g}.npy'.format(action), TD)

   
if __name__ == '__main__':
    user = np.arange(29) + 1
    lighting = [
        'fluorescent',
        'fluorescent_led',
        'lab',
        'led',
        'natural',
    ]
    count = 0
    for id in user:
        for light in lighting:
            filename = 'user{:02d}_{}'.format(id, light)
            if os.path.isfile(path + filename + '.aedat'):
                print(count, filename)
                splitData(filename, path)
                count += 1